In [12]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [13]:
# load dataset
all_data = pd.read_csv(r"../public/data.csv")

print(all_data.columns)
all_data.head()

Index(['file', 'utterance', 'DX1', 'MOCATOTS', 'MOCA_impairment', 'age',
       'gender', 'educ', 'tokens(participant)', 'uniquetokens(participant)',
       'AUX(participant)', 'CCONJ(participant)', 'NUM(participant)',
       'PROPN(participant)', 'VERB(participant)', 'DATE(participant)',
       'TTR(participant)', 'MATTR(participant)', 'REGTRYID', 'AB40_LUMI',
       'AB42_LUMI', 'P_TAU_LUMI', 'T_TAU_LUMI', 'AB42_AB40Ratio',
       'tTau_AB42Ratio', 'pTau_AB42Ratio'],
      dtype='object')


,file,utterance,DX1,MOCATOTS,MOCA_impairment,age,gender,educ,tokens(participant),uniquetokens(participant),...,TTR(participant),MATTR(participant),REGTRYID,AB40_LUMI,AB42_LUMI,P_TAU_LUMI,T_TAU_LUMI,AB42_AB40Ratio,tTau_AB42Ratio,pTau_AB42Ratio
0,3253_NormalText-INTERVIEW_ONLY_02-04-21I.edite...,"Okay. When I was a uh, senior in high school,...",Normal,21,0,88.0,0.0,20.0,375.000000,169.0,...,0.450135,0.994888,3253,11410.4,788.0,39.25,316.5,0.086684,0.327180,0.038540
1,3257_NormalText_INTERVIEW_ONLY_12-17-2020.edit...,I got off the plane and the first thing I not...,Normal,16,1,84.0,1.0,18.0,681.000000,269.0,...,0.395007,0.986730,3257,14892.0,691.0,111.00,674.0,0.046401,0.975398,0.160637
2,2764_NormalText-INTERVIEW_ONLY_10-15-20.edited...,okay this was when I was about 21 years old I ...,Normal,16,1,84.0,0.0,16.0,659.302326,248.5,...,0.371693,0.989751,2764,11410.4,788.0,39.25,316.5,0.086684,0.327180,0.038540
3,3351_NormalText_INTERVIEW_ONLY_07-28-20.edited...,"Well, the event I_x0080__x0099_d like to talk...",Normal,20,0,81.0,1.0,16.0,789.000000,291.0,...,0.368020,0.995052,3351,9487.0,637.0,39.10,303.0,0.067145,0.475667,0.061381
4,3373_NormalText_INTERVIEW_ONLY_03-11-21.edited...,When I was in college at San Luis Obispo at C...,Normal,19,0,82.0,1.0,18.0,538.000000,219.0,...,0.407063,0.996770,3373,14913.0,1421.0,47.10,329.0,0.095286,0.231527,0.033146


In [14]:
# extract relevant columns
# Notes: 
# Removed file, utterance, 'REGTRYID' because they are identifiers or not features needed for the ML model
relevant_columns = [
        'MOCATOTS', 'MOCA_impairment', 'DX1', 'age','gender', 'educ', 'tokens(participant)', 'uniquetokens(participant)',
       'AUX(participant)', 'CCONJ(participant)', 'NUM(participant)',
       'PROPN(participant)', 'VERB(participant)', 'DATE(participant)',
       'TTR(participant)', 'MATTR(participant)', 'AB40_LUMI',
       'AB42_LUMI', 'P_TAU_LUMI', 'T_TAU_LUMI', 'AB42_AB40Ratio',
       'tTau_AB42Ratio', 'pTau_AB42Ratio']


data = all_data[relevant_columns]
data.replace(['Normal', 'MCI', 'Prob AD'], [1, 2, 3], inplace=True) # change labels to numeric
data.head()

C:\Users\nandi\AppData\Local\Temp\ipykernel_166700\3736494479.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(['Normal', 'MCI', 'Prob AD'], [1, 2, 3], inplace=True) # change labels to numeric
C:\Users\nandi\AppData\Local\Temp\ipykernel_166700\3736494479.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace(['Normal', 'MCI', 'Prob AD'], [1, 2, 3], inplace=True) # change labels to numeric


,MOCATOTS,MOCA_impairment,DX1,age,gender,educ,tokens(participant),uniquetokens(participant),AUX(participant),CCONJ(participant),...,DATE(participant),TTR(participant),MATTR(participant),AB40_LUMI,AB42_LUMI,P_TAU_LUMI,T_TAU_LUMI,AB42_AB40Ratio,tTau_AB42Ratio,pTau_AB42Ratio
0,21,0,1,88.0,0.0,20.0,375.000000,169.0,20.000000,23.0,...,1.0,0.450135,0.994888,11410.4,788.0,39.25,316.5,0.086684,0.327180,0.038540
1,16,1,1,84.0,1.0,18.0,681.000000,269.0,46.000000,40.0,...,5.0,0.395007,0.986730,14892.0,691.0,111.00,674.0,0.046401,0.975398,0.160637
2,16,1,1,84.0,0.0,16.0,659.302326,248.5,47.732558,35.0,...,4.0,0.371693,0.989751,11410.4,788.0,39.25,316.5,0.086684,0.327180,0.038540
3,20,0,1,81.0,1.0,16.0,789.000000,291.0,45.000000,65.0,...,3.0,0.368020,0.995052,9487.0,637.0,39.10,303.0,0.067145,0.475667,0.061381
4,19,0,1,82.0,1.0,18.0,538.000000,219.0,32.000000,26.0,...,1.0,0.407063,0.996770,14913.0,1421.0,47.10,329.0,0.095286,0.231527,0.033146


In [15]:
# None of the features below are linguistic features which will be provided as input, so we will exclude them
exclude_features = ['age', 'gender', 'educ', 'AB40_LUMI', 'AB42_LUMI', 'P_TAU_LUMI', 'T_TAU_LUMI', 'AB42_AB40Ratio',
       'tTau_AB42Ratio', 'pTau_AB42Ratio', 'MOCATOTS', 'MOCA_impairment', 'DX1']

X = data.drop(columns = exclude_features, axis =1) # features
print(X.columns) # confirm if all features are linguistic features
y = data['DX1'] # label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

print(X_train.shape)
print(X_test.shape)

Index(['tokens(participant)', 'uniquetokens(participant)', 'AUX(participant)',
       'CCONJ(participant)', 'NUM(participant)', 'PROPN(participant)',
       'VERB(participant)', 'DATE(participant)', 'TTR(participant)',
       'MATTR(participant)'],
      dtype='object')
(60, 10)
(30, 10)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=1234)
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[24  0  3]
 [ 2  0  0]
 [ 1  0  0]]
Classification Report:
              precision    recall  f1-score   support

           1       0.89      0.89      0.89        27
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1

    accuracy                           0.80        30
   macro avg       0.30      0.30      0.30        30
weighted avg       0.80      0.80      0.80        30



c:\Users\nandi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\nandi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\nandi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

In [19]:
import joblib

joblib_file = "logistic_regression_model.joblib"
joblib.dump(lr_model, joblib_file)

['logistic_regression_model.joblib']